In [1]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import SparkSession
import pandas as pd
import time

In [2]:
json_path = 'C:/Files/College/GP/UCSDBooks/Downloads/goodreads_reviews_dedup.json'
csv_path = '../preprocessed_original/reviewsFull.csv'
bookidsPath = '../preprocessed_original/book_ids.csv'

columns = ['book_id', 'user_id', 'rating']
json_data = []

In [3]:
spark = SparkSession.builder.appName("Read JSON").getOrCreate()
spark

In [4]:
chunk_size = 100000
count = 0 
df = spark.read.json(json_path)


In [5]:
dfBooks = spark.read.csv(bookidsPath, header=True)
dfBooks.show()

+--------+
| book_id|
+--------+
| 2767052|
|       3|
|   41865|
|    2657|
|    4671|
|11870085|
|    5907|
|    5107|
|    1885|
|     960|
|   48855|
|    5470|
|13335037|
|    7613|
|       5|
| 2429135|
| 6148028|
|   77203|
|   15881|
|      34|
+--------+
only showing top 20 rows



In [6]:
df.show()

+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
| book_id|          date_added|        date_updated|n_comments|n_votes|rating|             read_at|           review_id|         review_text|          started_at|             user_id|
+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|24375664|Fri Aug 25 13:55:...|Mon Oct 09 08:55:...|         0|     16|     5|Sat Oct 07 00:00:...|5cd416f3efc3f944f...|Mind blowingly co...|Sat Aug 26 00:00:...|8842281e1d1347389...|
|18245960|Sun Jul 30 07:44:...|Wed Aug 30 00:00:...|         1|     28|     5|Sat Aug 26 12:05:...|dfdbb7b0eb5a7e4c2...|This is a special...|Tue Aug 15 13:23:...|8842281e1d1347389...|
| 6392944|Mon Jul 24 02:48:...|Sun Jul 30 09:28:...|         0|      6|     3|Tu

In [7]:

# select only user_id, book_id, rating columns
df = df.select(col("book_id"), col("user_id"), col("rating"), col("review_text"))
df.show()

+--------+--------------------+------+--------------------+
| book_id|             user_id|rating|         review_text|
+--------+--------------------+------+--------------------+
|24375664|8842281e1d1347389...|     5|Mind blowingly co...|
|18245960|8842281e1d1347389...|     5|This is a special...|
| 6392944|8842281e1d1347389...|     3|I haven't read a ...|
|22078596|8842281e1d1347389...|     4|Fun, fast paced, ...|
| 6644782|8842281e1d1347389...|     4|A fun book that g...|
| 1995421|8842281e1d1347389...|     0|Kevin highly reco...|
| 9460786|8842281e1d1347389...|     4|Giving a high rat...|
|29983426|8842281e1d1347389...|     5|I decided to give...|
|29893493|8842281e1d1347389...|     5|I haven't read a ...|
|28114110|8842281e1d1347389...|     4|Kevin Kelly, who ...|
|   16981|8842281e1d1347389...|     3|Recommended by Do...|
| 8191070|8842281e1d1347389...|     5|Best book of the ...|
|28684704|8842281e1d1347389...|     3|A fun, fast paced...|
|27161156|8842281e1d1347389...|     0|Re

In [8]:
df.count()
# if book id is not in book_ids.csv, remove it and merge book_id column with book_ids.csv
df = df.join(dfBooks, df.book_id == dfBooks.book_id, how='inner').select(df.user_id, dfBooks.book_id, df.rating, df.review_text)
# show count of rows
df.count()
df.show()


+--------------------+--------+------+--------------------+
|             user_id| book_id|rating|         review_text|
+--------------------+--------+------+--------------------+
|b77f34246ac69b4b6...|10000761|     3|A fascinating rea...|
|3ec16d3d84ccd6e92...|10000761|     5|Jason Berry has d...|
|6cff11a0530eeddeb...|10000761|     3|Interesting subje...|
|a45fb5d39a6a9857f...|10000761|     3|The author addres...|
|e4c2c0760e1b57734...| 1002783|     4|Lots of useful in...|
|ef2ccb0190bb3c0e7...| 1002783|     4|This was a useful...|
|bea77de12f3f6aac9...| 1002783|     5|An outstanding bo...|
|294a81d154d40b9d2...|10029674|     3|quite a good litt...|
|2b49259a6b753ce40...|10029674|     4|The Boy Who Playe...|
|6305fa58820260714...| 1003397|     4|Just a cute and f...|
|092e45a8ba9cf4548...|10046808|     5|One of the nicest...|
|59c0224b905ad2543...|10051877|     4|Shows a lot of po...|
|ca1c301fff032671a...|  100553|     5|We've seen hints ...|
|de6dd5aab1e5277cc...|  100553|     5|20

In [10]:
# write to single file
df.coalesce(1).write.csv(csv_path, header=True)
